# 1. Data

## 1.1. Read Data

In [15]:
# Import the libraries 
import time
import random
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from collections import defaultdict
from queue import PriorityQueue

In [16]:
df = pd.read_csv("Distance.csv")
df.head()

,Province,Ho Chi Minh,Ha Noi,Hai Phong,Can Tho,Dong Nai,Quang Nam,Bac Ninh,Hai Duong,Nghe An,...,Ha Nam,Hung Yen,Kien Giang,Lai Chau,Da Nang,Dak Nong,Dien Bien,Vinh Phuc,Latitude,Longitude
0,Ho Chi Minh,0.000000,1160.328125,1116.327582,124.340009,30.558650,597.455091,1160.851404,1132.378709,885.694721,...,1094.960639,1124.167884,253.245571,1338.581425,612.139008,197.376884,1284.528508,1184.499927,10.7638,106.6436
1,Ha Noi,1160.328125,0.000000,126.385860,1235.554616,1144.991818,656.690777,59.499238,88.532125,274.868404,...,75.458333,61.937428,1253.715745,264.688208,632.876926,1012.092866,258.969150,25.156596,21.1382,105.5050
2,Hai Phong,1116.327582,126.385860,0.000000,1201.305114,1098.305041,575.045991,74.855725,38.212372,257.613414,...,81.046226,67.797153,1233.502621,388.365117,552.142680,954.684700,385.354979,131.855404,20.8000,106.6667
3,Can Tho,124.340009,1235.554616,1201.305114,0.000000,154.836491,706.318330,1240.919573,1214.553251,961.075782,...,1173.594679,1203.685561,141.402052,1397.101956,719.138646,318.316161,1340.060030,1260.217144,10.0333,105.7833
4,Dong Nai,30.558650,1144.991818,1098.305041,154.836491,0.000000,572.923473,1144.164266,1115.131849,870.755857,...,1078.756363,1107.678552,282.901262,1327.577244,588.201480,169.248401,1274.383073,1168.998580,10.9277,106.8681


## 1.2. Random Main Warehouse

In [17]:
# Random mainwarehouse: mw_north, mw_central, mw_south
# Create list store province bound -> random
list_mw_north, list_mw_central, list_mw_south = [], [], []
for i in range(df.shape[0]):
    if df.iloc[i,64] > 20:
        list_mw_north.append(df.iloc[i,0])
    elif df.iloc[i,64] < 12:
        list_mw_south.append(df.iloc[i,0])
    else: 
        list_mw_central.append(df.iloc[i,0])

# Random
random.seed(100)
mw_north, mw_central, mw_south = random.choice(list_mw_north), random.choice(list_mw_central), random.choice(list_mw_south)
print(mw_north,mw_central,mw_south, sep = "\n")

Nam Dinh
Phu Yen
Long An


## 1.3. Convert Distance -> Distance Truck -> Distance Plane (Main Warehouse)

In [8]:
# Convert distance -> cost
# Cost truck = (Distance / 60) * 35
def distance_to_truck_cost(dis_truck):
    return (dis_truck * 35) / 60 
# apply to convert distance to cost truck 
df.iloc[:, 1:64] = df.iloc[:, 1:64].apply(distance_to_truck_cost)

In [9]:
# Check data after convert distance to cost
df.iloc[:,:]

,Province,Ho Chi Minh,Ha Noi,Hai Phong,Can Tho,Dong Nai,Quang Nam,Bac Ninh,Hai Duong,Nghe An,...,Ha Nam,Hung Yen,Kien Giang,Lai Chau,Da Nang,Dak Nong,Dien Bien,Vinh Phuc,Latitude,Longitude
0,Ho Chi Minh,0.000000,676.858073,651.191089,72.531672,17.825879,348.515470,677.163319,660.554247,516.655254,...,638.727039,655.764599,147.726583,780.839165,357.081088,115.136516,749.308296,690.958291,10.763800,106.643600
1,Ha Noi,676.858073,0.000000,73.725085,720.740193,667.911894,383.069620,34.707889,51.643740,160.339903,...,44.017361,36.130166,731.334185,154.401455,369.178207,590.387505,151.065337,14.674681,21.138200,105.505000
2,Hai Phong,651.191089,73.725085,0.000000,700.761317,640.677941,335.443495,43.665840,22.290550,150.274492,...,47.276965,39.548339,719.543196,226.546318,322.083230,556.899408,224.790404,76.915652,20.800000,106.666700
3,Can Tho,72.531672,720.740193,700.761317,0.000000,90.321287,412.019026,723.869751,708.489396,560.627540,...,684.596896,702.149911,82.484531,814.976141,419.497543,185.684427,781.701684,735.126667,10.033300,105.783300
4,Dong Nai,17.825879,667.911894,640.677941,90.321287,0.000000,334.205359,667.429155,650.493579,507.940916,...,629.274545,646.145822,165.025736,774.420059,343.117530,98.728234,743.390126,681.915838,10.927700,106.868100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Lai Chau,780.839165,154.401455,226.546318,814.976141,774.420059,521.504767,183.381359,204.294674,280.000815,...,195.620594,190.528976,811.832449,0.000000,507.441867,707.412782,38.626733,149.653438,22.368662,103.311913
59,Da Nang,357.081088,369.178207,322.083230,419.497543,343.117530,14.062900,357.682211,337.238944,230.279396,...,325.657531,339.111627,462.371261,507.441867,0.000000,248.701790,486.653225,380.969702,16.054407,108.202164
60,Dak Nong,115.136516,590.387505,556.899408,185.684427,98.728234,238.801955,586.631002,568.461533,432.743312,...,549.854495,565.935816,250.210789,707.412782,248.701790,0.000000,678.892727,603.904767,12.264648,107.609802
61,Dien Bien,749.308296,151.065337,224.790404,781.701684,743.390126,500.686568,183.650115,202.663746,256.936557,...,187.825169,186.362878,776.696085,38.626733,486.653225,678.892727,0.000000,149.986934,21.804232,103.107651


In [10]:
# Find id mw
(id_north, id_central, id_south) = (0,0,0)
for i in range(63):
    for j in range(63):
        for k in range(63):
            if (df.iloc[i,0] == mw_north) & (df.iloc[j,0] == mw_central) & (df.iloc[k,0] == mw_south):
                id_north += i
                id_central += j
                id_south += k
                
# Cost plane = (Distance / 80) * 160 = (120 * Distance_Truck) / 35
# North
df.at[id_north, mw_central] = df.at[id_north, mw_central] * 120 / 35
df.at[id_north, mw_south] = df.at[id_north, mw_south] * 120 / 35
# Central
df.at[id_central, mw_north] = df.at[id_north, mw_north] * 120 / 35
df.at[id_central, mw_south] = df.at[id_north, mw_south] * 120 / 35
# South
df.at[id_south, mw_north] = df.at[id_north, mw_north] * 120 / 35
df.at[id_south, mw_central] = df.at[id_north, mw_central] * 120 / 35

In [11]:
# Check data after convert distance to cost
df.iloc[:,:]

,Province,Ho Chi Minh,Ha Noi,Hai Phong,Can Tho,Dong Nai,Quang Nam,Bac Ninh,Hai Duong,Nghe An,...,Ha Nam,Hung Yen,Kien Giang,Lai Chau,Da Nang,Dak Nong,Dien Bien,Vinh Phuc,Latitude,Longitude
0,Ho Chi Minh,0.000000,676.858073,651.191089,72.531672,17.825879,348.515470,677.163319,660.554247,516.655254,...,638.727039,655.764599,147.726583,780.839165,357.081088,115.136516,749.308296,690.958291,10.763800,106.643600
1,Ha Noi,676.858073,0.000000,73.725085,720.740193,667.911894,383.069620,34.707889,51.643740,160.339903,...,44.017361,36.130166,731.334185,154.401455,369.178207,590.387505,151.065337,14.674681,21.138200,105.505000
2,Hai Phong,651.191089,73.725085,0.000000,700.761317,640.677941,335.443495,43.665840,22.290550,150.274492,...,47.276965,39.548339,719.543196,226.546318,322.083230,556.899408,224.790404,76.915652,20.800000,106.666700
3,Can Tho,72.531672,720.740193,700.761317,0.000000,90.321287,412.019026,723.869751,708.489396,560.627540,...,684.596896,702.149911,82.484531,814.976141,419.497543,185.684427,781.701684,735.126667,10.033300,105.783300
4,Dong Nai,17.825879,667.911894,640.677941,90.321287,0.000000,334.205359,667.429155,650.493579,507.940916,...,629.274545,646.145822,165.025736,774.420059,343.117530,98.728234,743.390126,681.915838,10.927700,106.868100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Lai Chau,780.839165,154.401455,226.546318,814.976141,774.420059,521.504767,183.381359,204.294674,280.000815,...,195.620594,190.528976,811.832449,0.000000,507.441867,707.412782,38.626733,149.653438,22.368662,103.311913
59,Da Nang,357.081088,369.178207,322.083230,419.497543,343.117530,14.062900,357.682211,337.238944,230.279396,...,325.657531,339.111627,462.371261,507.441867,0.000000,248.701790,486.653225,380.969702,16.054407,108.202164
60,Dak Nong,115.136516,590.387505,556.899408,185.684427,98.728234,238.801955,586.631002,568.461533,432.743312,...,549.854495,565.935816,250.210789,707.412782,248.701790,0.000000,678.892727,603.904767,12.264648,107.609802
61,Dien Bien,749.308296,151.065337,224.790404,781.701684,743.390126,500.686568,183.650115,202.663746,256.936557,...,187.825169,186.362878,776.696085,38.626733,486.653225,678.892727,0.000000,149.986934,21.804232,103.107651


In [12]:
# Save new file with convert to (truck, plane) cost
df.to_csv('Cost.csv', index = False)

##  1.4. Add Main Warehouse (cost) to Data Another Main Warehouse (3)

In [38]:
def data_A_star(start,goal, mw_north, mw_central, mw_south):
    data_neighbor ={'Lai Chau':['Dien Bien','Lao Cai','Son La','Yen Bai'],\
                'Yen Bai':['Lao Cai','Lai Chau','Son La','Ha Giang','Phu Tho','Tuyen Quang'],\
                'Dien Bien':['Lai Chau','Son La'],\
                'Thanh Hoa':['Son La','Nghe An','Ninh Binh','Hoa Binh'],
                'Nghe An':['Ha Tinh','Thanh Hoa'],\
                'Quang Binh':['Ha Tinh','Quang Tri'] ,\
                'Ha Tinh':['Nghe An','Quang Binh'],\
                'Quang Tri':['Quang Binh','Hue'],\
                'Hue':['Quang Tri','Da Nang','Quang Nam'],\
               'Da Nang':['Hue','Quang Nam'],\
                'Quang Nam':['Da Nang','Hue','Kon Tum','Quang Ngai'],\
                'Kon Tum':['Quang Nam','Quang Ngai','Gia Lai'],\
                'Quang Ngai':['Kon Tum','Quang Nam','Binh Dinh'],\
                'Gia Lai':['Kon Tum','Binh Dinh','Phu Yen','Dak Lak'],\
                'Binh Dinh':['Quang Ngai','Gia Lai','Phu Yen'],\
                'Dak Lak':['Dak Nong','Lam Dong','Khanh Hoa','Phu Yen','Gia Lai'],\
                'Phu Yen':['Gia Lai','Binh Dinh','Dak Lak','Khanh Hoa'],\
                'Khanh Hoa':['Dak Lak','Phu Yen','Lam Dong','Ninh Thuan'],\
                'Dak Nong':['Dak Lak','Lam Dong','Binh Phuoc'],\
                'Lam Dong':['Dak Lak','Dak Nong','Binh Phuoc','Dong Nai','Binh Thuan','Ninh Thuan','Khanh Hoa'],\
                'Ninh Thuan':['Lam Dong','Khanh Hoa','Binh Thuan'],\
                'Binh Phuoc':['Tay Ninh','Binh Duong','Dong Nai','Dak Nong','Lam Dong'],\
                'Dong Nai':['Lam Dong','Binh Phuoc','Binh Thuan','Ho Chi Minh','Vung Tau','Binh Duong'],\
                'Binh Thuan':['Vung Tau','Lam Dong','Ninh Thuan','Dong Nai'],\
                'Tay Ninh':['Long An', 'Ho Chi Minh', 'Binh Duong','Binh Phuoc'],\
                'Binh Duong':['Tay Ninh','Ho Chi Minh','Dong Nai','Binh Phuoc'],\
                'Ho Chi Minh':['Tay Ninh','Binh Duong','Dong Nai','Long An','Vung Tau'],\
                'Vung Tau':['Ho Chi Minh','Dong Nai','Binh Thuan'],\
                'Long An':['Tay Ninh','Ho Chi Minh','Dong Thap','Tien Giang'],\
                'Dong Thap':['Long An','Tien Giang','Vinh Long','Can Tho','An Giang'],\
                'Tien Giang':['Long An','Dong Thap','Vinh Long','Ben Tre'],\
                'Ben Tre':['Vinh Long','Tien Giang','Tra Vinh'],\
                'An Giang':['Kien Giang','Can Tho','Dong Thap'],\
                'Can Tho':['Kien Giang','An Giang','Dong Thap','Vinh Long','Hau Giang'],\
                'Vinh Long':['Can Tho','Dong Thap','Ben Tre','Tra Vinh','Hau Giang'],\
                'Tra Vinh':['Ben Tre','Vinh Long','Soc Trang'],\
                'Soc Trang':['Hau Giang','Bac Lieu','Tra Vinh'],\
                'Hau Giang':[ 'Kien Giang','Bac Lieu','Soc Trang','Can Tho','Vinh Long'],\
                'Kien Giang':['An Giang','Can Tho','Hau Giang','Bac Lieu','Ca Mau'],\
                'Bac Lieu':['Kien Giang','Hau Giang','Soc Trang','Ca Mau'],\
                'Ca Mau':['Kien Giang','Bac Lieu'],\
                'Ha Noi':['Phu Tho','Hoa Binh','Bac Giang','Vinh Phuc','Bac Ninh','Hung Yen','Ha Nam'],\
                'Phu Tho':['Son La','Yen Bai','Hoa Binh','Ha Noi','Vinh Phuc','Tuyen Quang'],\
                'Hoa Binh':['Ninh Binh','Thanh Hoa','Ha Nam','Ha Noi','Phu Tho','Son La'],\
                'Bac Giang':['Quang Ninh','Hai Duong','Bac Ninh','Ha Noi','Thai Nguyen','Lang Son'],\
                'Vinh Phuc':['Phu Tho', 'Ha Noi','Thai Nguyen','Tuyen Quang'],\
                'Bac Ninh':['Ha Noi','Hung Yen','Hai Duong', 'Bac Giang'],\
                'Hung Yen':['Ha Noi','Bac Ninh','Hai Duong','Thai Binh','Ha Nam'],\
                'Ha Nam':['Ha Noi','Hung Yen','Thai Binh','Nam Dinh','Ninh Binh','Hoa Binh'],\
                'Hai Phong':['Thai Binh','Thai Binh','Quang Ninh'],\
                'Hai Duong':['Bac Ninh', 'Hung Yen','Thai Binh','Hai Phong','Quang Ninh','Bac Giang'],\
                'Nam Dinh':['Ninh Binh','Ha Nam','Thai Binh'],\
                'Thai Nguyen':['Tuyen Quang', 'Bac Kan','Lang Son','Bac Giang','Ha Noi','Vinh Phuc'],\
                'Thai Binh':['Nam Dinh', 'Ha Nam','Hung Yen','Hai Duong','Hai Duong','Hai Phong'],\
                'Quang Ninh':['Hai Duong','Hai Phong','Bac Giang','Lang Son'],\
                'Lang Son':['Quang Ninh','Bac Giang','Thai Nguyen','Bac Kan','Cao Bang'],\
                'Ninh Binh':['Hoa Binh','Thanh Hoa','Nam Dinh','Ha Nam'],\
                'Lao Cai':['Lai Chau','Yen Bai','Ha Giang'],\
                'Cao Bang':['Ha Giang','Bac Kan','Lang Son'],\
                'Ha Giang':['Cao Bang','Tuyen Quang','Yen Bai','Lao Cai'],\
                'Tuyen Quang':['Ha Giang','Bac Kan','Thai Nguyen','Vinh Phuc','Phu Tho','Yen Bai'],\
                'Bac Kan':['Tuyen Quang','Thai Nguyen','Lang Son','Cao Bang'],\
                'Son La':['Dien Bien','Yen Bai','Phu Tho','Hoa Binh','Thanh Hoa'],\
               }
    df = pd.read_csv("Cost.csv")
    Province1 = df['Province']
    Province=[]
    for i in Province1:
        Province.append(i.strip())

    list_distance=[]
    for i in df.values:
        list_distance.append(list(i[1:-2]))
    dict_province={}
    for i in range(len(Province)):
        dict_province[Province[i]]=i
    dict_province    
    dict_data={}
    for i in data_neighbor:
        list_each_province=[]
        for j in data_neighbor[i]:
            list_each_province.append(j)
            list_each_province.append(list_distance[dict_province[i]][dict_province[j]])
        list
        list_each_province.append(list_distance[dict_province[i]][dict_province[goal]])
        dict_data[i]=list_each_province
        print(dict_data[i])
    return dict_data  

data_neigborhood = data_A_star('Ha Noi','Thanh Hoa', mw_north, mw_central, mw_south)
# Add data mainware house for North mainware house
#data_neigborhood[mw_north] = data_neigborhood[mw_north].insert(-2, mw_central) 
#print(data_neigborhood[mw_north])



['Dien Bien', 38.62673315083333, 'Lao Cai', 41.11098010833334, 'Son La', 76.702058125, 'Yen Bai', 103.08571443333334, 223.3141467666667]
['Lao Cai', 70.188872075, 'Lai Chau', 103.08571443333334, 'Son La', 62.38772131666667, 'Ha Giang', 73.22151229166667, 'Phu Tho', 40.552163248333336, 'Tuyen Quang', 22.60769014583333, 134.62626741666668]
['Lai Chau', 38.62673315083333, 'Son La', 57.678885407500005, 207.31970058333332]
['Son La', 150.05943767500003, 'Nghe An', 73.77204905, 'Ninh Binh', 31.39257597, 'Hoa Binh', 70.76569039166665, 0.0]
['Ha Tinh', 25.13445792083333, 'Thanh Hoa', 73.77204905, 73.77204905]
['Ha Tinh', 70.34965250833334, 'Quang Tri', 53.4672809825, 159.07551461666668]
['Nghe An', 25.13445792083333, 'Quang Binh', 70.34965250833334, 95.3469629]
['Quang Binh', 53.4672809825, 'Hue', 38.19853345, 210.92461325833327]
['Quang Tri', 38.19853345, 'Da Nang', 46.42659845833334, 'Quang Nam', 59.859344825, 244.09650055000003]
['Hue', 46.42659845833334, 'Quang Nam', 14.062899678333334, 28